RQ2: To what extent do non-trivial open-source repositories that implement both GUI and performance end-to-end tests differ from those that implement only GUI end-to-end tests or only performance end-to-end tests, with respect to project activity metrics such as the number of commits, contributors, issues, and pull requests?

### Imports

In [ ]:
import pandas as pd
from scipy import stats

### Load CSVs

In [ ]:
df_repository_general = pd.read_csv('E2EGit\\repository.csv')
df_repository_general = df_repository_general.rename(columns={'name': 'repository_name'})
df_repository_general = df_repository_general[['repository_name', 'commits','contributors', 'total_issues', 'total_pull_requests']]

df_filtered = df_repository_general[
    (df_repository_general['commits'] >= 2000) &
    (df_repository_general['contributors'] >= 10) &
    (df_repository_general['total_issues'] >= 100) &
    (df_repository_general['total_pull_requests'] >= 50)
].reset_index(drop=True)

df_filtered.head(10)

Load non_trivial_repository CSV

In [ ]:
df_repository_non_trivial = pd.read_csv('E2EGit\\non_trivial_repository.csv')

df_repository_non_trivial = df_repository_non_trivial.rename(columns={'name': 'repository_name'})

df_repository_non_trivial = df_repository_non_trivial[['repository_name']]

Merge general info with non trivial repos

In [ ]:
df_repository = pd.merge(df_filtered, df_repository_non_trivial, left_on='repository_name', right_on='repository_name', how='inner')

print(len(df_repository))

print(df_repository)

Clean up the data

In [ ]:
numeric_cols = ['commits', 'contributors', 'total_issues', 'total_pull_requests']

for col in numeric_cols:
    df_repository[col] = pd.to_numeric(df_repository[col], errors='coerce')

# drop NaNs
df_repository = df_repository.dropna(subset=numeric_cols)

print(df_repository)
print(len(df_repository))

Load GUI details CSV

In [ ]:
df_gui_repo_details = pd.read_csv('E2EGit\gui_testing_repo_details.csv')

# Keep only the columns you want
df_gui_repo_details = df_gui_repo_details[['repository_name']]

print(df_gui_repo_details)
print(len(df_gui_repo_details))

Load performance details CSV

In [ ]:
df_performance_test_details = pd.read_csv('E2EGit\performance_testing_test_details.csv')
df_performance_test_details = df_performance_test_details[['repository_name']]

print(df_performance_test_details)
print(len(df_performance_test_details))

Merge performance with GUI testing to get repositories that implement both

In [ ]:
df_both_tests = pd.merge(df_performance_test_details, df_gui_repo_details, left_on='repository_name', right_on='repository_name', how='inner')

print(df_both_tests)
print(len(df_both_tests))

Get reposorties that implement GUI only

In [ ]:
df_gui_only = pd.merge(df_gui_repo_details, df_performance_test_details, on='repository_name', how='left', indicator=True)
df_gui_only = df_gui_only[df_gui_only['_merge'] == 'left_only'][['repository_name']]

print(df_gui_only)
print(len(df_gui_only))

Get reposorties that implement Performance only

In [ ]:
df_perf_only = pd.merge(df_performance_test_details, df_gui_repo_details, on='repository_name', how='left', indicator=True)
df_perf_only = df_perf_only[df_perf_only['_merge'] == 'left_only'][['repository_name']]

print(df_perf_only)
print(len(df_perf_only))

Merge both Dataframe with the Dataframe that contains project activity

In [ ]:
repo_both_with_repository_details = pd.merge(df_both_tests, df_repository, on='repository_name', how='inner')

repo_both_with_repository_details.head(10)

Not merged repositories (exist in GUI or Performance but not in both) with repository details

In [ ]:
repo_gui_only_with_repository_details = pd.merge(df_gui_only, df_repository, on='repository_name', how='inner')
repo_perf_only_with_repository_details = pd.merge(df_perf_only, df_repository, on='repository_name', how='inner')

print(len(repo_gui_only_with_repository_details))
print(len(repo_perf_only_with_repository_details))


Add a column to each DataFrame to identify their type

In [ ]:
repo_both_with_repository_details['test_type'] = 'Both'
repo_gui_only_with_repository_details['test_type'] = 'GUI'
repo_perf_only_with_repository_details['test_type'] = 'Performance'

Get the final DataFrame

In [ ]:
df_all = pd.concat([repo_both_with_repository_details, repo_gui_only_with_repository_details, repo_perf_only_with_repository_details], ignore_index=True)

print(len(df_all))

###  Normality Tests

In [ ]:
normality_results = {}

for test_type in df_all['test_type'].unique():
    print(f"{test_type}:")
    subset = df_all[df_all['test_type'] == test_type]
    normality_results[test_type] = {}
    
    for metric in numeric_cols:
        stat, p_value = stats.shapiro(subset[metric])
        is_normal = p_value > 0.05
        normality_results[test_type][metric] = is_normal
        print(f"  {metric:15s}: W={stat:.4f}, p={p_value:.4f} | "
              f"{'Normal' if is_normal else 'Non normal'}")
    print()

In [ ]:
summary_stats = df_all.groupby('test_type').agg({
    'commits': ['mean', 'median', 'std', 'count'],
    'contributors': ['mean', 'median', 'std'],
    'total_issues': ['mean', 'median', 'std'],
    'total_pull_requests': ['mean', 'median', 'std']
}).round(2)

print(summary_stats)

### Mann-Whitney U test

In [ ]:
gui_only = df_all[df_all['test_type'] == 'GUI']
perf_only = df_all[df_all['test_type'] == 'Performance']
both_tests = df_all[df_all['test_type'] == 'Both']

comparisons = [
    ('Both', 'GUI Only', both_tests, gui_only),
    ('Both', 'Performance Only', both_tests, perf_only),
    ('GUI Only', 'Performance Only', gui_only, perf_only)
]

mw_results = {}

for metric in numeric_cols:
    print(f"{metric.upper()}")
    print("-" * 80)
    mw_results[metric] = {}
    
    for label1, label2, data1, data2 in comparisons:
        values1 = data1[metric]
        values2 = data2[metric]
        
        # Mann-Whitney U test
        u_stat, p_value = stats.mannwhitneyu(values1, values2, alternative='two-sided')
        
        # Calculate effect size (rank-biserial correlation)
        n1, n2 = len(values1), len(values2)
        rank_biserial = 1 - (2*u_stat) / (n1 * n2)
        
        # Calculate medians
        median1 = values1.median()
        median2 = values2.median()
        median_diff = median1 - median2
        pct_diff = (median_diff / median2) * 100 if median2 != 0 else 0
        
        sig_marker = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
        
        mw_results[metric][f"{label1}_vs_{label2}"] = {
            'u_stat': u_stat,
            'p_value': p_value,
            'rank_biserial': rank_biserial,
            'median_diff': median_diff,
            'pct_diff': pct_diff
        }
        
        print(f"  {label1} vs {label2}")
        print(f"    Median 1: {median1:.2f} | Median 2: {median2:.2f}")
        print(f"    Difference: {median_diff:+.2f} ({pct_diff:+.1f}%)")
        print(f"    U-statistic: {u_stat:.2f}")
        print(f"    P-value: {p_value:.4f} {sig_marker}")
        print(f"    Effect size (r): {rank_biserial:.3f}")
    
    print()